In [98]:
import numpy as np
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import ScriptRunConfig, RunConfiguration, Experiment
from azureml.train.estimator import Estimator
from azureml.data.data_reference import DataReference
#from azureml.data.dataset_consumption_config import DatasetConsumptionConfig 

In [84]:
ws = Workspace.from_config()

In [85]:
def_blob_store = Datastore(ws, "workspaceblobstore")
def_file_store = Datastore(ws, "workspacefilestore")

# Testing clean.py

In [87]:
dataset = Dataset.get_by_name(ws, name='annonces_ds')
experiment = Experiment(ws, "CleanTest")

clean_est = Estimator(source_directory='./pipeline_steps', entry_script='clean.py',
                script_params= {"--input": 'annonces_ds', "--output": 'cleantest'},
                inputs=[dataset.as_named_input('annonces_ds')],
                compute_target='local',
                use_docker=False
               )

run = experiment.submit(clean_est)

run.wait_for_completion(show_output=True)

RunId: CleanTest_1585823600_183b9301
Web View: https://ml.azure.com/experiments/CleanTest/runs/CleanTest_1585823600_183b9301?wsid=/subscriptions/68bdd703-8837-469c-80bd-bfb35f3b886f/resourcegroups/ProjectGroup2/workspaces/RealEstatePG2

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt

Streaming azureml-logs/70_driver_log.txt

Starting the daemon thread to refresh tokens in background for process with pid = 669
Entering Run History Context Manager.
Preparing to call script [ clean.py ] with arguments: ['--input', 'annonces_ds', '--output', 'cleantest']
After variable expansion, calling script [ clean.py ] with arguments: ['--input', 'annonces_ds', '--output', 'cleantest']

/home/jovyan/.azureml/envs/azureml_da3e97fcb51801118b8e80207f3e01ad/lib/python3.6/site-packages/azureml/dataprep/api/dataflow.py:722: UserWarning: Your pandas and pyarrow versions are incompatible. Please install pyarrow>=0.12.0 for improved performance of to_pandas_datafr

{'runId': 'CleanTest_1585823600_183b9301',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-04-02T10:33:21.356673Z',
 'endTimeUtc': '2020-04-02T10:34:42.11558Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '187c48e4-92c4-4d8b-9222-b9ec642f942d'},
 'inputDatasets': [{'dataset': {'id': '5a88c2bd-5165-485b-a81f-bc8290270b5a'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'annonces_ds', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'clean.py',
  'useAbsolutePath': False,
  'arguments': ['--input', 'annonces_ds', '--output', 'cleantest'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'annonces_ds': {'dataLocation': {'dataset': {'id': '5a88c2bd-5165-485b-a81f-bc8290270b5a'},
     'dataPath': None},
    'createOutputDirectories': False,
    'mechanism': 'Direct',
    'environmentVariableName': 'annonces_ds',
    'pathOnCompute

In [88]:
run.download_file('outputs/cleantest', output_file_path='./cleantmp.csv')
!head ./cleantmp.csv

,idannonce,typedebien,codepostal,idtypechauffage,idtypecuisine,si_balcon,nb_chambres,nb_pieces,si_sdbain,si_sdEau,etage,prix,surface,dpeC,date
0,154680523,Appartement,33000,gaz collectif,,0.0,2.0,3,1,0,2.0,843,60.0,0.0,2019-03-28 15:30:41.565724000
1,155037587,Appartement,33700,individuel électrique radiateur,aucune,0.0,0.0,1,0,0,2.0,512,21.35,195.0,2019-03-28 19:37:48.081944028
2,152350643,Appartement,33140,,,1.0,2.0,3,0,0,3.0,859,69.48,138.0,2019-03-28 23:44:54.598164056
3,155513075,Appartement,33400,,,1.0,1.0,2,0,0,3.0,695,47.25,162.0,2019-03-29 03:52:01.114384084
4,155025785,Appartement,33000,gaz collectif,,0.0,2.0,3,1,0,4.0,814,68.0,0.0,2019-03-29 07:59:07.630604112
5,155551059,Appartement,33700,individuel électrique,américaine,1.0,2.0,3,1,0,0.0,756,65.51,0.0,2019-03-29 12:06:14.146824141
7,154854333,Appartement,33600,,équipée,0.0,1.0,2,0,1,0.0,720,36.0,0.0,2019-03-29 20:20:27.179264197
8,155325225,Appartement,33600,individuel,,1.0,1.0,2,0,1,1.0,564,37.01,167.0,2019-03-30 00:27:33

In [108]:
clean_ref = def_blob_store.upload_files(
    ['./cleantmp.csv'],
    target_path='tmp/clean.csv',
    overwrite=True)
clean_ref.data_reference_name = "clean_data"
clean_ref.mode = 'download' # 'download'
clean_ref

Uploading an estimated of 1 files
Uploading ./cleantmp.csv
Uploaded ./cleantmp.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_clean_data

In [110]:
datastore_paths = [(def_blob_store, 'tmp/clean.csv')]
clean_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)
# # clean_ds = clean_ds.register(workspace=ws,
# #                            name='clean_ds',
# #                            description='annonces data clean')
# clean_ds = DatasetConsumptionConfig('clean_ds', clan_ds, mode='direct', path_on_compute=None)

# Testing split.py

In [116]:
split_est = Estimator(source_directory='./pipeline_steps',
                      entry_script='split.py',
                      script_params= {"--dataset": "clean_data",
                                      "--train": "train_ds",
                                      "--valid": "valid_ds",
                                      "--trainsize": 400,
                                      "--validsize": 100},
                      #inputs=[clean_ref],
                      inputs=[dataset.as_named_input('clean_data')],
                      compute_target='local',
                      use_docker=False
                     )

run = experiment.submit(split_est)

run.wait_for_completion(show_output=True)

RunId: CleanTest_1585827071_4c6083c2
Web View: https://ml.azure.com/experiments/CleanTest/runs/CleanTest_1585827071_4c6083c2?wsid=/subscriptions/68bdd703-8837-469c-80bd-bfb35f3b886f/resourcegroups/ProjectGroup2/workspaces/RealEstatePG2

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 1962
Running: ['/bin/bash', '/tmp/azureml_runs/CleanTest_1585827071_4c6083c2/azureml-environment-setup/conda_env_checker.sh']
Found materialized conda environment in: /home/jovyan/.azureml/envs/azureml_da3e97fcb51801118b8e80207f3e01ad


Running: ['/home/jovyan/.azureml/envs/azureml_da3e97fcb51801118b8e80207f3e01ad/bin/python', 'azureml-setup/run_script.py', '/home/jovyan/.azureml/envs/azureml_da3e97fcb51801118b8e80207f3e01ad/bin/python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'RunHistory:context_managers.Run

{'runId': 'CleanTest_1585827071_4c6083c2',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-04-02T11:31:13.3316Z',
 'endTimeUtc': '2020-04-02T11:32:33.615598Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'd15f9fe6-61d6-4004-b6d8-796d9f34f37b'},
 'inputDatasets': [{'dataset': {'id': '5a88c2bd-5165-485b-a81f-bc8290270b5a'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'clean_data', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'split.py',
  'useAbsolutePath': False,
  'arguments': ['--dataset',
   'clean_data',
   '--train',
   'train_ds',
   '--valid',
   'valid_ds',
   '--trainsize',
   '400',
   '--validsize',
   '100'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'clean_data': {'dataLocation': {'dataset': {'id': '5a88c2bd-5165-485b-a81f-bc8290270b5a'},
     'dataPath': None},
    'createOutputDirectories': False,
 

In [117]:
run.download_file('outputs/train_ds', output_file_path='./train_dstmp.csv')
!head ./cleantmp.csv
run.download_file('outputs/valid_ds', output_file_path='./valid_dstmp.csv')
!head ./cleantmp.csv

,idannonce,typedebien,codepostal,idtypechauffage,idtypecuisine,si_balcon,nb_chambres,nb_pieces,si_sdbain,si_sdEau,etage,prix,surface,dpeC,date
0,154680523,Appartement,33000,gaz collectif,,0.0,2.0,3,1,0,2.0,843,60.0,0.0,2019-03-28 15:30:41.565724000
1,155037587,Appartement,33700,individuel électrique radiateur,aucune,0.0,0.0,1,0,0,2.0,512,21.35,195.0,2019-03-28 19:37:48.081944028
2,152350643,Appartement,33140,,,1.0,2.0,3,0,0,3.0,859,69.48,138.0,2019-03-28 23:44:54.598164056
3,155513075,Appartement,33400,,,1.0,1.0,2,0,0,3.0,695,47.25,162.0,2019-03-29 03:52:01.114384084
4,155025785,Appartement,33000,gaz collectif,,0.0,2.0,3,1,0,4.0,814,68.0,0.0,2019-03-29 07:59:07.630604112
5,155551059,Appartement,33700,individuel électrique,américaine,1.0,2.0,3,1,0,0.0,756,65.51,0.0,2019-03-29 12:06:14.146824141
7,154854333,Appartement,33600,,équipée,0.0,1.0,2,0,1,0.0,720,36.0,0.0,2019-03-29 20:20:27.179264197
8,155325225,Appartement,33600,individuel,,1.0,1.0,2,0,1,1.0,564,37.01,167.0,2019-03-30 00:27:33

In [118]:
train_ref = def_blob_store.upload_files(
    ['./train_dstmp.csv'],
    target_path='tmp/train.csv',
    overwrite=True)
train_ref.data_reference_name = "train_data"
train_ref.mode = 'download'

valid_ref = def_blob_store.upload_files(
    ['./valid_dstmp.csv'],
    target_path='tmp/valid.csv',
    overwrite=True)
valid_ref.data_reference_name = "valid_data"
valid_ref.mode = 'download'

Uploading an estimated of 1 files
Uploading ./train_dstmp.csv
Uploaded ./train_dstmp.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading ./valid_dstmp.csv
Uploaded ./valid_dstmp.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [119]:
datastore_paths = [(def_blob_store, 'tmp/train.csv')]
train_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)

datastore_paths = [(def_blob_store, 'tmp/valid.csv')]
valid_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)

# Testing train.py

In [ ]:
split_est = Estimator(source_directory='.', entry_script='./pipeline_steps/train.py',
                script_params= {"--dataset": "train_data",
                                "--model": "model.pkl"},
                inputs=[train_ref],
                #inputs=[dataset.as_named_input('clean_data')],
                compute_target='local',
                use_docker=False
               )

run = experiment.submit(split_est)

run.wait_for_completion(show_output=True)

In [ ]:
model = run.register_model(model_name='test_model',
                           tags={'test': 'test'},
                           model_path='outputs/model.pkl')

# Testing eval.py

In [ ]:
eval_est = Estimator(source_directory='.', entry_script='./pipeline_steps/eval.py',
                script_params= {"--dataset": "valid_data",
                                "--model": "model.pkl"},
                inputs=[valid_ref],
                compute_target='local',
                use_docker=False
               )

run = experiment.submit(eval_est)

run.wait_for_completion(show_output=True)